In [1]:
import robotic as ry
import numpy as np
import time

# Part A

In [2]:
C = ry.Config()
C.addFile("Gfiles-HW2/Question-01/environment/cargobot.g")
C.addFile("Gfiles-HW2/Question-01/environment/cargo.g")
C.addFile("Gfiles-HW2/Question-01/environment/maze.g")
C.view()

0

In [3]:
# Get and set the home joint configuration
qHome_first = C.getJointState()
C.setJointState(qHome_first)

In [4]:
# Initialize KOMO with 1 phase, 10 steps, and a 2-second duration
komo1 = ry.KOMO(C, 2, 10, 2, False)

In [5]:
# Define control objectives for movement smoothness and speed
komo1.addControlObjective([], 0, 1e-1)
komo1.addControlObjective([], 2, 1e0)

In [6]:
# Set objectives to approach, position, and orient the gripper relative to the cargo
komo1.addObjective([1], ry.FS.positionDiff, ['l_panda_base', 'cargo'], ry.OT.eq, [1e1])
komo1.addObjective([2], ry.FS.positionDiff, ['l_gripper', 'cargo_handle'], ry.OT.eq, [1e1])
komo1.addObjective([2], ry.FS.vectorZ, ['l_gripper'], ry.OT.eq, [1e1], [0, 0, 1])

In [7]:
# Apply joint limits constraint
komo1.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq, [1e1])

In [8]:
# Solve the KOMO problem and retrieve the path
ret = ry.NLP_Solver(komo1.nlp(), verbose=0).solve()
print(ret)

q1 = komo1.getPath()
print('size of path:', q1.shape)

# Visualize each waypoint in the computed trajectory
for t in range(q1.shape[0]):
    C.setJointState(q1[t])
    C.view(False, f'waypoint {t}')
    time.sleep(0.1)

{ time: 0.015625, evals: 28, done: 1, feasible: 0, sos: 19.065, f: 0, ineq: 0, eq: 0.700581 }
size of path: (20, 10)


In [9]:

partajs = q1[-1]
print("penetration:", C.getCollisionsTotalPenetration())

penetration: 0.0


### PART B

In [10]:
# Start Task 2 from the end state of Task 1
C.setJointState(partajs)


In [11]:
# Attach the cargo to the gripper for transport
C.attach("l_gripper", "cargo")

In [12]:
# Set up KOMO for Task 2 with similar parameters as Task 1
komo2 = ry.KOMO(C, 2, 10, 2, False)

In [13]:
# Control objectives for smooth and controlled motion
komo2.addControlObjective([], 0, 1e-1)
komo2.addControlObjective([], 2, 1e0)

In [14]:
# Objective to transport cargo to goal area and align orientation
komo2.addObjective([2], ry.FS.positionDiff, ['cargo', 'goal_area'], ry.OT.eq, [1e1])
komo2.addObjective([2], ry.FS.vectorZ, ['l_gripper'], ry.OT.eq, [1e1], [0, 0, 1])

In [15]:
# Joint limits to avoid collisions and ensure safety
komo2.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq, [1e1])

In [16]:
# Solve Task 2 and retrieve the path
ret2 = ry.NLP_Solver(komo2.nlp(), verbose=0).solve()
print(ret2)

q2 = komo2.getPath()
print('Size of path:', q2.shape)

{ time: 0, evals: 36, done: 1, feasible: 1, sos: 15.7217, f: 0, ineq: 0, eq: 5.79615e-05 }
Size of path: (20, 10)


In [17]:
# Display each waypoint in the path to ensure it is collision-free
for t in range(q2.shape[0]):
    C.setJointState(q2[t])
    C.view(False, f'waypoint {t}')
    time.sleep(0.1)

In [18]:
# Final state and detach the cargo
task2js = q2[-1]
print("Penetration:", C.getCollisionsTotalPenetration())
C.getFrame("cargo").unLink()

Penetration: 0.0


# Part B

## Cargo Movement

In [19]:
C = ry.Config()
C.addFile("Gfiles-HW2/Question-01/environment/cargobot.g")
C.addFile("Gfiles-HW2/Question-01/environment/cargo.g")
C.addFile("Gfiles-HW2/Question-01/environment/maze.g")
rrt = ry.PathFinder()
rrt.setProblem(C, [qHome_first], [partajs])
ret = rrt.solve()
print(ret)
pathCargo = ret.x
for t in range(0, pathCargo.shape[0]-1):
    C.setJointState(pathCargo[t])
    C.view()
    time.sleep(.1)

{ time: 0.1875, evals: 4171, done: 1, feasible: 1, sos: -1, f: -1, ineq: -1, eq: -1 }


## To the Goal Movement

In [20]:
C.attach("l_gripper", "cargo")
rrt2 = ry.PathFinder()
rrt2.setProblem(C, [partajs], [task2js])
ret = rrt2.solve()
print(ret)
pathGoal = ret.x
for t in range(0, pathGoal.shape[0]-1):
    C.setJointState(pathGoal[t])
    C.view()
    time.sleep(.1)

{ time: 0.0625, evals: 726, done: 1, feasible: 1, sos: -1, f: -1, ineq: -1, eq: -1 }


# Part C

In [42]:
C = ry.Config()

In [43]:
C.addFile("Gfiles-HW2/Question-01/environment/cargobot.g")
C.addFile("Gfiles-HW2/Question-01/environment/cargo.g")
C.addFile("Gfiles-HW2/Question-01/environment/maze.g")

In [44]:
C.view()

0

In [45]:
time.sleep(2)

In [46]:
for t in range(0, pathCargo.shape[0]-1):
    C.setJointState(pathCargo[t])
    C.view()
    time.sleep(.1)
C.attach("l_gripper", "cargo")

In [47]:
for t in range(0, pathGoal.shape[0]-1):
    C.setJointState(pathGoal[t])
    C.view()
    time.sleep(.1)
C.getFrame("cargo").unLink()
C.view()

0